# Reproducing Results

In [57]:
from transformers import pipeline

print(pipeline("sentiment-analysis")("we love you"))


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'POSITIVE', 'score': 0.9998704195022583}]


In [58]:
from transformers import GPT2Tokenizer, GPT2Model

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [59]:
model = GPT2Model.from_pretrained('gpt2', output_hidden_states=True)

In [60]:
model
model.eval()

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP

In [61]:
tokenized = tokenizer(['The rain in Spain falls mainly in the plains.'], return_tensors='pt')

In [62]:
tokenized.keys

<bound method BatchEncoding.keys of {'input_ids': tensor([[  464,  6290,   287,  8602,  8953,  8384,   287,   262, 36149,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}>

In [63]:
tokenized['input_ids'].shape

torch.Size([1, 10])

In [64]:
tokenizer.convert_ids_to_tokens(tokenized['input_ids'][0])

['The',
 'Ġrain',
 'Ġin',
 'ĠSpain',
 'Ġfalls',
 'Ġmainly',
 'Ġin',
 'Ġthe',
 'Ġplains',
 '.']

In [65]:
model.wte(tokenized['input_ids']).shape
model.wte(tokenized['input_ids'])

tensor([[[-0.0686, -0.0203,  0.0645,  ...,  0.0639, -0.0100,  0.0031],
         [ 0.0997, -0.0837, -0.0261,  ...,  0.0033,  0.0892, -0.1988],
         [-0.0337,  0.0108,  0.0293,  ...,  0.1106, -0.0134, -0.0528],
         ...,
         [-0.0393,  0.0050,  0.0421,  ..., -0.0477,  0.0670, -0.0471],
         [ 0.0261,  0.0570,  0.0442,  ...,  0.0725, -0.0586, -0.0349],
         [ 0.0466, -0.0113,  0.0283,  ..., -0.0735,  0.0496,  0.0963]]],
       grad_fn=<EmbeddingBackward0>)

In [66]:
import torch
with torch.no_grad():
    output = model(**tokenized)
    print(len(output.hidden_states))
    print(output.hidden_states[0].shape)
    print(output.hidden_states[1].shape)
#     print(output.last_hidden_state.shape)
#     print(output.last_hidden_state)

13
torch.Size([1, 10, 768])
torch.Size([1, 10, 768])


In [67]:
# ...last_hidden_state[:, -1, :]

with torch.no_grad():
    print(torch.mean(model(**tokenizer(['The cat in the hat'], return_tensors='pt')).last_hidden_state, dim=1).shape)

torch.Size([1, 768])


In [68]:
model = GPT2Model(config=model.config).eval()
print(model.config)

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "output_hidden_states": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.12.5",
  "use_cache": true,
  "vocab_size": 50257
}

